In [1]:
# convlstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
   


Using TensorFlow backend.


In [2]:
pip install keras-video-generators

  Created wheel for keras-video-generators: filename=keras_video_generators-1.0.13-cp36-none-any.whl size=12729 sha256=11e78017269b5c405259c0950972941040c5adc93d8cd7e08dc9927a72918691
  Stored in directory: /root/.cache/pip/wheels/f4/9a/15/fa18c9caaeb672d7deb2b5be938fcf0c29b259c0cf40aa43f7
Successfully built keras-video-generators


In [0]:
import os
import glob
from keras_preprocessing import image
from keras_video import VideoFrameGenerator
# use sub directories names as classes
import keras
classes = [i.split(os.path.sep)[6] for i in glob.glob('/content/drive/My Drive/PROJECT_DATA/Hands_only_OpenPose/*')]
classes.sort()


In [4]:
print(classes)

['friday', 'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday']


In [0]:
SIZE = (120, 120)
CHANNELS = 3
NBFRAME = 5
BS = 5

In [50]:
glob_pattern='/content/drive/My Drive/PROJECT_DATA/Hands_only_OpenPose/{classname}/*.mp4'
# for data augmentation
print(list(classes), len(classes))

['friday', 'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday'] 7


In [0]:
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)


In [52]:
train = VideoFrameGenerator(
    classes=classes,
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.20,
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=True)

class friday, validation count: 20, train count: 82
class monday, validation count: 23, train count: 92
class saturday, validation count: 20, train count: 83
class sunday, validation count: 25, train count: 103
class thursday, validation count: 12, train count: 49
class tuesday, validation count: 26, train count: 106
class wednesday, validation count: 11, train count: 47
Total data: 7 classes for 562 files for train


In [44]:
valid = train.get_validation_generator()

Total data: 7 classes for 137 files for validation


In [0]:
import keras_video.utils
#keras_video.utils.show_sample(train)
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D
from keras.layers import TimeDistributed, GRU, Dense, Dropout

In [0]:
EPOCHS=10
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        '/content/drive/My Drive/chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1)
]


In [0]:
a = len(classes)
def action_model(a):
  model = Sequential()
  model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(5, 120,120,3)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(a, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit_generator(train, epochs=EPOCHS, validation_data= valid, verbose=1, callbacks=callbacks)
	
#, batch_size=BS
  # evaluate model
	# _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)


In [0]:
#INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 224, 224, 3)
model = action_model(len(classes))
optimizer = keras.optimizers.SGD()

Epoch 1/10
 58/112 [==============>...............] - ETA: 3:56 - loss: 1.9153 - acc: 0.1966

In [0]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [0]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y



In [0]:
 
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [0]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	# define model 
  Frame,size, Shape - 12,120,120,3
	verbose, epochs, batch_size = 0, 25, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape into subsequences (samples, time steps, rows, cols, channels)
	n_steps, n_length = 4, 32
	trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
	# define model
	model = Sequential()
	model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
	model.add(Dropout(0.5))
	model.add(keras.layers.Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy



In [0]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [0]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)
 



In [0]:
# run the experiment
run_experiment()